In [1]:
#filter warnings
import warnings
warnings.filterwarnings('ignore')

#to read data
import pandas as pd
import numpy as np

#term frequencey- inverse document frequncy is a numerical statistic that is intended to reflect how important a word is to document in a collecion or corpus
from sklearn.feature_extraction.text import TfidfVectorizer

#for computation of tfidf matrix
from sklearn.metrics.pairwise import linear_kernel

#to calaculation of similarity measure
from sklearn.metrics.pairwise import cosine_similarity

**1) Build a recommender system with the given data using UBCF.
This dataset is related to the video gaming industry and a survey was conducted to build a 
recommendation engine so that the store can improve the sales of its gaming DVDs. Snapshot of the dataset is given below. Build a Recommendation Engine and suggest top selling DVDs to the store customers.**

In [2]:
data1=pd.read_csv(r'E:\Study Materials DATA SCIENCE\360digitmg\Assingment Folder\Assingment 12 Recommendation Engine\Datasets_Recommendation Engine\game.csv')
data1.head()

,userId,game,rating
0,3,The Legend of Zelda: Ocarina of Time,4.0
1,6,Tony Hawk's Pro Skater 2,5.0
2,8,Grand Theft Auto IV,4.0
3,10,SoulCalibur,4.0
4,11,Grand Theft Auto IV,4.5


In [3]:
#Creating a sparse matrix 
matrix = data1.pivot_table( columns='game', index='userId', values='rating')
#it will have many nan value

#remove some of the gamers who don’t play that many games
matrix = matrix.dropna(thresh=2, axis=0) #code snippet basically counts those who have less than 2 games played and drops their rows
matrix

game,'Splosion Man,007: The World is Not Enough,10 Second Ninja X,1001 Spikes,1701 A.D.,1979 Revolution: Black Friday,2002 FIFA World Cup,2010 FIFA World Cup South Africa,3D After Burner II,3D Gunstar Heroes,...,echochrome,echochrome ii,escapeVektor: Chapter 1,flower,ilomilo,inFamous,inFamous 2,inFamous: Festival of Blood,inFamous: Second Son,page not found
userId,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#These are the most high rated or popular games
data1.groupby('game')['rating'].mean().sort_values(ascending=False).head(10)

game
'Splosion Man                           5.0
Marvel Pinball                          5.0
Crimson Skies: High Road to Revenge     5.0
Crisis Core: Final Fantasy VII          5.0
Critter Crunch                          5.0
Skies of Arcadia Legends                5.0
Mercury Meltdown                        5.0
Sins of a Solar Empire: Entrenchment    5.0
SingStar                                5.0
SimCity 4                               5.0
Name: rating, dtype: float64

In [5]:
#fill the null value with zeros
matrix = data1.pivot_table(columns='game', index='userId', values='rating', fill_value=0)

#standardize the row values
def std(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row
matrix_std = matrix.apply(std)

In [6]:
def gameRec(g):
    dota = matrix_std[g] #The dota variable is what I used to return our values from our centered matrix for a particular game
    
    #Calculate correlation 
    dota = matrix.corrwith(dota).dropna()
    #create a DF to show how many times each game has been played and the mean rating has been given
    gameData = data1.groupby('game').agg({'rating': [np.size, np.mean]})
    #Filter out any game played by less than 100 players.
    gameSim = gameData['rating']['size'] >= 5
    df = gameData[gameSim].join(pd.DataFrame(dota, columns=['similarity']))
    return df.sort_values(['similarity'], ascending=False)[:7]

In [7]:
gameRec('Marvel Pinball')

,"(rating, size)","(rating, mean)",similarity
game,,,
Shovel Knight,5.0,3.3,-0.000629
Tiger Woods PGA Tour 07,5.0,3.2,-0.000656
Bit.Trip Presents...Runner2: Future Legend of Rhythm Alien,5.0,3.7,-0.000659
Tony Hawk's Pro Skater 2,5.0,4.0,-0.000664
Rayman Origins,5.0,4.0,-0.000670
FIFA Soccer 09,5.0,3.6,-0.000670
Tony Hawk's Pro Skater 4,5.0,3.9,-0.000672


In [8]:
gameRec("Tony Hawk's Pro Skater 2")

,"(rating, size)","(rating, mean)",similarity
game,,,
Tony Hawk's Pro Skater 2,5.0,4.0,1.000000
Shovel Knight,5.0,3.3,-0.001362
Tiger Woods PGA Tour 07,5.0,3.2,-0.001420
Bit.Trip Presents...Runner2: Future Legend of Rhythm Alien,5.0,3.7,-0.001426
Rayman Origins,5.0,4.0,-0.001449
FIFA Soccer 09,5.0,3.6,-0.001450
Tony Hawk's Pro Skater 4,5.0,3.9,-0.001455


**2. The Entertainment Company, which is an online movie watching platform, wants to improve its collection of movies and showcase those that are highly rated and recommend those movies to its customer by their movie watching footprint. For this, the company has collected the data and shared it with you to provide some analytical insights and also to come up with a recommendation algorithm so that it can automate its process for effective recommendations. The ratings are between -9 and +9.**

In [24]:
data2=pd.read_csv(r'E:\Study Materials DATA SCIENCE\360digitmg\Assingment Folder\Assingment 12 Recommendation Engine\Datasets_Recommendation Engine\Entertainment.csv', encoding = 'utf8')
data2.head()

,Id,Titles,Category,Reviews
0,6973,Toy Story (1995),"Drama, Romance, School, Supernatural",-8.98
1,6778,Jumanji (1995),"Action, Adventure, Drama, Fantasy, Magic, Mili...",8.88
2,9702,Grumpier Old Men (1995),"Action, Comedy, Historical, Parody, Samurai, S...",99.00
3,6769,Waiting to Exhale (1995),"Sci-Fi, Thriller",99.00
4,1123,Father of the Bride Part II (1995),"Action, Comedy, Historical, Parody, Samurai, S...",-0.44


In [25]:
tfidf=TfidfVectorizer(stop_words="english")
tfidf #it is a statistical measure that evaluates how relevant a word is to a document in a collection of documents

TfidfVectorizer(stop_words='english')

In [26]:
# Preparing the Tfidf matrix by fitting and transforming
tfidf_matrix= tfidf.fit_transform(data2.Category)
tfidf_matrix.shape

(51, 34)

In [31]:
# For now we will be using cosine similarity matrix
cosine_sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

data_index = pd.Series(data2.index, index = data2['Titles']).drop_duplicates()
data_index["Jumanji (1995)"]

1

In [35]:
def get_recommendations(Name, topN):    
    # topN = 10
    # Getting the movie index using its title 
    movie_id = data_index[Name]
    
    # Getting the pair wise similarity score for all the movies's with that 
    # anime
    cosine_scores = list(enumerate(cosine_sim_matrix[movie_id]))
    
    # Sorting the cosine_similarity scores based on scores x[1]
    cosine_scores = sorted(cosine_scores, key=lambda x:x[1], reverse = True)
    
    # Get the scores of top N most similar movies 
    cosine_scores_N = cosine_scores[0: topN+1]
    
    # Getting the movie index 
    movie_idx  =  [i[0] for i in cosine_scores_N]
    movie_scores =  [i[1] for i in cosine_scores_N]
    
    # Similar movies and scores
    movie_similar_show = pd.DataFrame(columns=["name", "Score"])
    movie_similar_show["name"] = data2.loc[movie_idx, "Titles"]
    movie_similar_show["Score"] = movie_scores
    movie_similar_show.reset_index(inplace = True)  
    
    # movie_similar_show.drop(["index"], axis=1, inplace=True)
    print (movie_similar_show)
    # return (movie_similar_show)
    

In [36]:
get_recommendations('Jumanji (1995)', topN=10)

    index                                  name     Score
0       1                        Jumanji (1995)  1.000000
1      24              Leaving Las Vegas (1995)  0.537426
2      35                       Clueless (1995)  0.426853
3      15                         Casino (1995)  0.343851
4       6                        Sabrina (1995)  0.320244
5      13                          Nixon (1995)  0.318409
6       7                   Tom and Huck (1995)  0.314619
7      22                      Assassins (1995)  0.275029
8      11    Dracula: Dead and Loving It (1995)  0.267170
9      36       Cry, the Beloved Country (1995)  0.265075
10     42  How to Make an American Quilt (1995)  0.264094
